<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent-Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain Agent Insurance - using OpenAI functions

Different agents and code implementations how to run Langchain agents using OpenAI functions: Single prompt, chat with memory, from command line or in Gradio web app, using AgentExecutor or your own custom loop.

## How to use
1. Run all cells of group "Basic setup for all agents, tools and apps"
2. Run one of the two agent/tool groups
3. Run different code implementations after that: CLI, Gradio, custom loops...

For more information see https://levelup.gitconnected.com/building-a-powerful-agent-has-no-challenge-today-774e27be818d

# Basic setup for all agents, tools and apps

In [1]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [2]:
!pip install langchain==0.0.311 langchain-experimental==0.0.27 langchainhub==0.1.13 google-search-results==2.4.2 wikipedia==1.4.0 openai==0.28.1 gradio==3.47.1 -q
%load_ext gradio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 

In [3]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.llms import OpenAI
from langchain.chains.llm import LLMChain
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool, StructuredTool
from langchain.schema.agent import AgentFinish
from langchain.schema import BaseMessage, AIMessage, HumanMessage
import os
import langchain
import openai
import textwrap
import gradio as gr
import random
import time

# Agent 2: Calculate insurance premium

In [23]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential']
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')

def calculate_insurance_premium(insured_sum: int, currency: str, industry: str, customer: str = "", country: str = "Germany") -> float:
    """
    Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.
    Examples:
    - Calculate the premium for an insured sum of 20000000 EUR, industry is construcion, customer "Miller Ltd."
    - Calculate an offer for Bayer AG, chemical industriy, insured sum 1000000 USD
    - Give me a premium for chemical industriy, sum 1000000 M$
    - Give me an offer for contruction industry and 20000000 TEUR insured sum
    - Calculate with insured sum 10000000 EUR
    """
    premium = insured_sum*0.25
    result = str(premium) + " (for " + country + ")"
    return result

calc_tool = StructuredTool.from_function(calculate_insurance_premium, return_direct=False)

def show_calculation_history() -> str:
    """
    Show the history of all calcuations to the user. Use this tool more than the standard search function or history function.
    Examples:
    - Show me the history of my calculations
    - Show me all my offers
    """
    chat_history_string = str(chat_history).replace("), ", ")§§§ ")[1:-1]
    chat_history_array = chat_history_string.split("§§§ ")
    msg_history_string = ""
    for chat_string in chat_history_array:
      tmp = chat_string.split("(content='")
      msg_history_string = msg_history_string + tmp[0] + " - " + tmp[1][:-2] + "\n"
    return msg_history_string

show_calculation_history_tool = StructuredTool.from_function(show_calculation_history, return_direct=True)

def show_hints() -> str:
    """
    Show hints for using this agent. Use only these texts Use the same language as the answer before..
    """
    hint = "Hint: Ask 'Show me all customer names and premiums, use CSV format and ; as delimiter' "
    return hint

show_hints_tool = StructuredTool.from_function(show_hints, return_direct=True)

tools = [calc_tool, show_calculation_history_tool]
tools_as_openai_functions = [format_tool_to_openai_function(t) for t in tools]
tools_dict = {
    "calculate_insurance_premium": calc_tool,
    "show_calculation_history": show_calculation_history_tool,
    #"show_hints": show_hints_tool
    }

tools_string = ""
for t in tools:
  tools_string = tools_string + " - " + str(t) + "\n"


tools_string = """ - name='calculate_insurance_premium' description='Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.'
 - name='show_calculation_history' description='Show the history of all calcuations to the user. Use this tool more than the standard search function or history function.' """

llm_with_tools = llm.bind(
    functions=tools_as_openai_functions
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant. Use the following tools:\n" + tools_string),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"],
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


print(tools_string)

 - name='calculate_insurance_premium' description='Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.'
 - name='show_calculation_history' description='Show the history of all calcuations to the user. Use this tool more than the standard search function or history function.' 


In [ ]:
# @title
# show calc_tool as OpenAI function
print(json.dumps(format_tool_to_openai_function(calc_tool), sort_keys=False, indent=2))

{
  "name": "calculate_insurance_premium",
  "description": "calculate_insurance_premium(insured_sum: int, industry: str, country: str = 'Germany') -> float - Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.",
  "parameters": {
    "title": "calculate_insurance_premiumSchemaSchema",
    "type": "object",
    "properties": {
      "insured_sum": {
        "title": "Insured Sum",
        "type": "integer"
      },
      "industry": {
        "title": "Industry",
        "type": "string"
      },
      "country": {
        "title": "Country",
        "default": "Germany",
        "type": "string"
      }
    },
    "required": [
      "insured_sum",
      "industry"
    ]
  }
}


In [ ]:
# @title
demoSchema = {
  "name": "calculate_insurance_premium",
  "description": "calculate_insurance_premium(insured_sum: int, industry: str, country: str = 'Germany') -> float - Calculate the premium for an insurance based on the maximum insured sum and the industry of the customer.",
  "parameters": {
    "title": "calculate_insurance_premiumSchemaSchema",
    "type": "object",
    "properties": {
      "insured_sum": {
        "title": "Insured Sum",
        "type": "integer"
      },
      "industry": {
        "title": "Industry",
        "type": "string"
      },
      "country": {
        "title": "Country",
        "default": "Germany",
        "type": "string",
        "enum": ["positive", "negative"],
        "description": "The Country where the insured company is located"
      }
    },
    "required": [
      "insured_sum",
      "industry"
    ]
  }
}

# Chat loop

In [24]:
langchain.debug = False

intermediate_steps = []
chat_history = []

block_memory = False

# manual loop / like AgentExecutor
def chat_manual_loop(prompt_text: str) -> str:
  block_memory = False
  while True:
      output = agent.invoke({
          "input": prompt_text, # + "\nGib anschliessend einen Tipp zur Nutzung dieses Agents aus.",
          "intermediate_steps": intermediate_steps,
          "chat_history": chat_history
      })
      if isinstance(output, AgentFinish):
          result = output.return_values["output"]
          if block_memory == False:
            chat_history.append(HumanMessage(content=(prompt_text)))
            chat_history.append(AIMessage(content=(output.return_values["output"])))
          break
      else:
          print(output.tool, output.tool_input)
          if output.tool=="show_history":
            block_memory = False # True
            print("*** memory blocked")
          else:
            block_memory = False
          tool = tools_dict[output.tool]
          observation = tool.run(output.tool_input)
          intermediate_steps.append((output, observation))
  block_memory == False
  #result = result + "\n\n => [Hier] weitere Tipps und Infos zur Nutzung."
  return result

#chat_response = chat_manual_loop("hi")
chat_response = chat_manual_loop("Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000 EUR")
print(chat_response)

Um die Versicherungsprämie zu berechnen, benötige ich weitere Informationen. Könnten Sie mir bitte die Branche und den Kunden (falls vorhanden) nennen?


In [25]:
chat_response = chat_manual_loop("Die Branche ist Metallbau.")
print(chat_response)

calculate_insurance_premium {'insured_sum': 10000000, 'currency': 'EUR', 'industry': 'Metallbau'}
Die Versicherungsprämie für die Metallbau-Branche mit einer Versicherungssumme von 10.000.000 EUR beträgt 2.500.000 EUR (für Deutschland).


In [31]:
chat_response = chat_manual_loop("Show me all customer names and premiums, use CSV format and ; as delimiter")
print(chat_response)

show_calculation_history {}
Hier sind alle Kundennamen und Prämien in CSV-Format:

"Kunde";"Prämie"
"Metallbau-Branche";"2.500.000 EUR"
"Müller GmbH";"1.250.000 EUR"
"Müller GmbH";"500.000 EUR"
"Schweizer AG";"1.250.000 EUR"


In [29]:
#chat_response = chat_manual_loop("Fasse alle letzten Nachrichten in einem Satz zusammen.")
chat_response = chat_manual_loop("Liste die letzten Berechnungen auf.")
#chat_response = chat_manual_loop("Using Custom Tool show me the history of my messages")
print(chat_response)

Hier sind Ihre letzten Berechnungen:

1. Die Versicherungsprämie für die Metallbau-Branche mit einer Versicherungssumme von 10.000.000 EUR beträgt 2.500.000 EUR (für Deutschland).
2. Die Versicherungsprämie für die Müller GmbH in der Software-Branche mit einer Versicherungssumme von 5.000.000 EUR beträgt 1.250.000 EUR (für Deutschland).
3. Die Versicherungsprämie für die Müller GmbH in der Software-Branche mit einer Versicherungssumme von 2.000.000 EUR beträgt 500.000 EUR (für Deutschland).


In [ ]:
chat_history_string = str(chat_history).replace("), ", ")§§§ ")[1:-1]
print(chat_history_string)
chat_history_array = chat_history_string.split("§§§ ")
msg_history_string = ""
for chat_string in chat_history_array:
  tmp = chat_string.split("(content='")
  msg_history_string = msg_history_string + tmp[0] + " - " + tmp[1][:-2] + "\n"
print(msg_history_string)

HumanMessage(content='Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000')§§§ AIMessage(content='Um die Versicherungsprämie zu berechnen, benötige ich weitere Informationen. Könnten Sie mir bitte die Währung, die Branche und das Land (falls nicht Deutschland) angeben?')§§§ HumanMessage(content='EUR, Die Branche ist Metallbau.')§§§ AIMessage(content='Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 EUR in der Metallbaubranche beträgt 2.500.000 EUR (für Deutschland).')
HumanMessage - Kalkuliere die Versicherungsprämie für eine Versicherungssumme von 10000000
AIMessage - Um die Versicherungsprämie zu berechnen, benötige ich weitere Informationen. Könnten Sie mir bitte die Währung, die Branche und das Land (falls nicht Deutschland) angeben?
HumanMessage - EUR, Die Branche ist Metallbau.
AIMessage - Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 EUR in der Metallbaubranche beträgt 2.500.000 EUR (für Deutschland).



In [30]:
chat_response = chat_manual_loop("Berechne für Schweizer AG, Versicherungssumme von 5 MEUR, Branche Software")
print(chat_response)

show_calculation_history {}
calculate_insurance_premium {'insured_sum': 5000000, 'currency': 'EUR', 'industry': 'Software', 'customer': 'Schweizer AG'}
Die Versicherungsprämie für die Schweizer AG in der Software-Branche mit einer Versicherungssumme von 5.000.000 EUR beträgt 1.250.000 EUR (für Deutschland).


In [27]:
chat_response = chat_manual_loop("Berechne erneut, ändere die Versicherungssumme auf 2000000 EUR")
print(chat_response)

calculate_insurance_premium {'insured_sum': 2000000, 'currency': 'EUR', 'industry': 'Software', 'customer': 'Müller GmbH'}
Die Versicherungsprämie für die Müller GmbH in der Software-Branche mit einer Versicherungssumme von 2.000.000 EUR beträgt 500.000 EUR (für Deutschland).


In [ ]:
chat_response = chat_manual_loop("Liste die letzten namentlich bekannten Kunden auf, bitte keine Dopplungen. Gib zusätzlich die angebotene Prämie an. Gib das Ergebnis im CSV-Format zurück, Trennzeichen ;")
print(chat_response)

Hier sind die letzten namentlich bekannten Kunden und die angebotene Prämie:

1. Müller GmbH; 2500000.0 EUR

Tipp zur Nutzung dieses Agents: Fragen Sie 'Zeigen Sie mir alle Kundennamen und Prämien, verwenden Sie das CSV-Format und ; als Trennzeichen'.


In [ ]:
show_history()

IndexError: ignored

In [ ]:
print("Memory from response object:")
chat_history_string = str(response["chat_history"]).replace("), ", ")§§§ ")[1:-1]
chat_history = chat_history_string.split("§§§ ")
msg_history = []
for chat_string in chat_history:
  tmp = chat_string.split("(content='")
  msg_history.append({"role": tmp[0], "content": tmp[1][:-2]})
#for msg in msg_history:
#  print(msg)
print('\n'.join(str(x) for x in msg_history))

Memory from response object:
{'role': 'HumanMessage', 'content': 'Give me the premium for an insured sum of 10000000'}
{'role': 'AIMessage', 'content': 'Sure, I can help with that. However, I also need to know the industry of the customer to calculate the insurance premium. Could you please provide that information?'}
{'role': 'HumanMessage', 'content': 'The industry is chemical production'}
{'role': 'AIMessage', 'content': 'The premium for an insured sum of 10,000,000 in the chemical production industry in Germany is 2,500,000.'}


# Gradio Chat App for Agent

In [ ]:
# Init agent and memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)
messages = [{"role": "assistant", "content": "How can I help you?"}]

#
# Gradio app
#

# Theming
theme = gr.themes.Default(
    primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
)
# Styling: Change max width
css = """
  .gradio-container {max-width: 800px!important}
  .vspacer1 {margin-top: 20px}
"""

with gr.Blocks(theme=theme, css=css) as demo:

    gr.Markdown("# Agent Chat", elem_classes="vspacer1")

    with gr.Tab("Chat"):
      # https://www.gradio.app/docs/chatbot

      chatbot = gr.Chatbot(bubble_full_width=False)
      msg = gr.Textbox()
      clear = gr.ClearButton([msg, chatbot])

      def ask(message, chat_history):
          chat_history.append((message, None))
          messages.append({"role": "user", "content": message})
          return "", chat_history

      def respond(chat_history):
          prompt = chat_history[-1][0] # get prompt from history (last entry)
          response = agent_executor.invoke({"input": prompt})
          msg = {"role": "assistant", "content": response["output"]}
          messages.append(msg)
          chat_history.append((None, response["output"]))
          print("\n\nMemory from response object:")
          print(textwrap.fill(str(response["chat_history"]), 80))
          return chat_history

      msg.submit(ask, [msg, chatbot], [msg, chatbot], queue=False).then(
        respond, chatbot, chatbot
      )

demo.launch(quiet=True, share=False, debug=True)

<IPython.core.display.Javascript object>



> Entering new AgentExecutor chain...
Um die Versicherungsprämie zu berechnen, benötige ich weitere Informationen. Könnten Sie mir bitte die Währung, die Branche und das Land (falls nicht Deutschland) angeben?

> Finished chain.


Memory from response object:
[HumanMessage(content='Kalkuliere die Versicherungsprämie für eine
Versicherungssumme von 10000000'), AIMessage(content='Um die Versicherungsprämie
zu berechnen, benötige ich weitere Informationen. Könnten Sie mir bitte die
Währung, die Branche und das Land (falls nicht Deutschland) angeben?')]


> Entering new AgentExecutor chain...

Invoking: `calculate_insurance_premium` with `{'insured_sum': 10000000, 'currency': 'EUR', 'industry': 'Metallbau'}`


2500000.0 (for Germany)Die Versicherungsprämie für eine Versicherungssumme von 10.000.000 EUR in der Metallbau-Branche beträgt 2.500.000 EUR in Deutschland.

> Finished chain.


Memory from response object:
[HumanMessage(content='Kalkuliere die Versicherungsprämie für eine
Versiche